- 지니차트 top200 chart

In [1]:
import requests
import pandas as pd

### 인터넷에서 데이터 가져오기

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [4]:
# Chrome User-Agent
header = {'User-Agent':
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}
req = requests.get(url, headers=header)
html = req.text
#html

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

### 찾으려고 하는 데이터의 노드 찾기

In [7]:
trs = soup.find_all('tr')
len(trs)

51

In [9]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [13]:
# 개발자 도구에서 찾아주는 selector
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
trs = tbody.select('tr')
len(trs)

50

- XPath 값은 아래와 같으나 BeautifupSoup은 지원하지 않음 
<pre>
//*[@id="body-content"]/div[6]/div/table/tbody<pre>

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- rank, title, artist, album

In [31]:
tr = trs[0]

In [16]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [18]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [21]:
# title
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [23]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [26]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text()
artist

"BE'O (비오)"

In [29]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text()
album

'쇼미더머니 10 Episode 3'

### 4. 한페이지에 있는 모든 데이터를 반복문으로 가져와 데이터 프레임 만들기

In [32]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    rank_list.append(rank), 
    title_list.append(title), 
    artist_list.append(artist), 
    album_list.append(album)
    

In [35]:
df = pd.DataFrame({
    '순위':rank_list, '제목':title_list, '가수':artist_list, '앨범':album_list
})
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [40]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    lines.append([rank, title, artist, album])

df = pd.DataFrame(lines, columns =['순위', '제목', '가수', '앨범'])
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [43]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    d = {'순위':rank, '제목':title, '가수':artist, '앨범':album}
    lines.append(d)

df = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


### 5. 모든 페이지의 데이터 가져오기

In [47]:
Lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text()
        album = tr.select_one('.albumtitle.ellipsis').get_text()
        d = {'순위':rank, '제목':title, '가수':artist, '앨범':album}
        Lines.append(d)

df = pd.DataFrame(Lines)
df.tail()
    

,순위,제목,가수,앨범
195,196,＋82 (Prod. by 다이나믹 듀오 & THAMA),팔로알토 (Paloalto) & 기리보이 & 다이나믹 듀오 & 저스디스 (JUSTH...,쇼미더머니 10 Final
196,197,DINOSAUR,AKMU (악뮤),SUMMER EPISODE
197,198,추적이는 여름 비가 되어,장범준,장범준 싱글
198,199,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2 (tvN 목요드라마)
199,200,밤하늘의 별을 (Original),임한별,밤하늘의 별을 (Original)


In [48]:
df.to_csv('지니차트_Top200(21.12.06).csv', index=False)